In [1]:
import pandas as pd
import numpy as np
import random as rd

import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.ml.feature import VectorSlicer
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession

import findspark
findspark.init()

spark = SparkSession\
    .builder\
    .appName("PCA com spark")\
    .getOrCreate()

def sliceFeatures(df, nValores, nomeSaidas, nomeEntrada):#dataframe / numero de colunas a serem criadas / nome das colunas de saida / ...coluna de entrada
    for i in range(nValores):
        slicer = VectorSlicer(inputCol = nomeEntrada, outputCol = nomeSaidas + str(i+1), indices=[i])
        df = slicer.transform(df)
    return df


def returnVector(df, name):#dataframe / nome da coluna , retorna um vetor da coluna especificada
    aux = np.array(df.select(name).collect())
    aux2 = []
    for i in range(df.count()):
        aux2.append(float(aux[i]))
    return aux2



def returnDimesion(nAmostras, nTipos, rangeDaMedia):#retorna um vetor np com nAmostras, com nTipos com a mesma media, que varia de 1 a rangeDaMedia 
    d = []
    for k in range(nTipo):
        t = np.random.poisson(lam=rd.randrange(1, rangeDaMedia), size = nAmostras)
        d = np.concatenate((d, t))
    return d

In [2]:
nComponentes = 4         #quantas componentes vão ser calculadas (não precisa mudar) (tem que ser maior que 3)#*********************************

#FEATURES
dimensoes = ['sepal length','sepal width','petal length','petal width']

color = ['blue', 'orange', 'green', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan', 'black', 'yellow', 'white']# Cor de cada classe

classe = False# Seus dados tem classes?

tags = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'] #Classes de amostras

coluna_tags = 'variety' #Coluna onde estão as classes

caminho = "/home/erick/Bases/iris.csv"#caminho do arquivo
caminho = "iris-Copy1.csv"#caminho do arquivo

In [3]:
dataframe = spark.read.format("csv").options(sep=',',header='true',inferschema='true').load(caminho)
dataframe.dtypes

Py4JJavaError: An error occurred while calling o58.load.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3, 10.129.64.70, executor 1): java.io.FileNotFoundException: File file:/home/erick/Redução com PCA/iris-Copy1.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:124)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:169)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.s

In [ ]:
assembler = VectorAssembler(inputCols = dimensoes, outputCol="features")  # junta as colunas em uma coluna features
dataframe = assembler.transform(dataframe) #aplicando a transformacao

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)
scalerModel = scaler.fit(dataframe)                     #cria uma coluna scaledFeatures que é a features com media 0 e desvio padrão 1
scaledData = scalerModel.transform(dataframe)

pandas = scaledData.toPandas()
pandas

In [ ]:
pca = PCA(k = nComponentes, inputCol="scaledFeatures", outputCol="pcaFeatures") #cria uma coluna pcaFeatures com a imagem das amostras nas PC's 
model = pca.fit(scaledData)
result = model.transform(scaledData)

df = sliceFeatures(result,  nComponentes, 'PC-', 'pcaFeatures') #separa a coluna pcaFeatures em colunas PC-X

pandas = df.toPandas()
pandas

In [ ]:
per_var = np.round(model.explainedVariance * 100, decimals = 1)#quanta informação cada pc tem em %, o ultimo é zero pq ne (não sei pq ele entra)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]# labels para o grafico
plt.bar(x=range(1,len(per_var)+1), height=per_var, tick_label=labels)#grafico de barras
plt.ylabel('Contribuição na variancia dos dados')
plt.xlabel('Componente Principal')
plt.title('Importancia de Cada componente')
plt.show()

In [ ]:
if classe:
    for i in range(len(tags)):# cada iteração printa um a cor
        pc1 = np.array(df.select('PC-1', coluna_tags).filter(df.   label    == tags[i]).drop(coluna_tags).collect())#df. (trocar coluna tags pelo nome da coluna onde estão as classes)
        pc2 = np.array(df.select('PC-2', coluna_tags).filter(df.   label    == tags[i]).drop(coluna_tags).collect())#df. (trocar coluna tags pelo nome da coluna)
        print('In {} the {}'.format(color[i], tags[i]))
        plt.scatter(pc1, pc2, color = color[i]) # color
else:
    pc1 = np.array(df.select('PC-1').collect())#df. (trocar coluna tags pelo nome da coluna)
    pc2 = np.array(df.select('PC-2').collect())#df. (trocar coluna tags pelo nome da coluna)
    plt.scatter(pc1, pc2, color = color[rd.randrange(1, 10)]) # color

plt.title('Grafico com PC1 e PC2')
plt.xlabel('PC1 - {0}%'.format(per_var[0]))
plt.ylabel('PC2 - {0}%'.format(per_var[1]))
plt.show()

In [ ]:
#Agora com PC1, PC2 e PC3
import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

pc1 = returnVector(df, 'PC-1')
pc2 = returnVector(df, 'PC-2')
pc3 = returnVector(df, 'PC-3')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel('PC1 - {0}%'.format(per_var[0]))
ax.set_ylabel('PC2 - {0}%'.format(per_var[1]))
ax.set_zlabel('PC3 - {0}%'.format(per_var[2]))

if classe:
    for i in range(len(tags)):# cada iteração printa um a cor
        pc1 = np.array(df.select('PC-1', coluna_tags).filter(df. label == tags[i]).drop(coluna_tags).collect())#df. (trocar coluna tags pelo nome da coluna)
        pc2 = np.array(df.select('PC-2', coluna_tags).filter(df. label == tags[i]).drop(coluna_tags).collect())#df. (trocar coluna tags pelo nome da coluna)
        pc3 = np.array(df.select('PC-3', coluna_tags).filter(df. label == tags[i]).drop(coluna_tags).collect())#df. (trocar coluna tags pelo nome da coluna)
        print('In {} the {}'.format(color[i], tags[i]))
        ax.scatter(pc1, pc2, pc3,color = color[i]) # OBS: Em geral a escala de PC3 é bem menor que a dos outros eixos
else:
    pc1 = np.array(df.select('PC-1').collect())#df. (trocar coluna tags pelo nome da coluna)
    pc2 = np.array(df.select('PC-2').collect())#df. (trocar coluna tags pelo nome da coluna)
    pc3 = np.array(df.select('PC-3').collect())#df. (trocar coluna tags pelo nome da coluna)
    ax.scatter(pc1, pc2, pc3,color = color[rd.randrange(1, 10)]) # OBS: Em geral a escala de PC3 é bem menor que a dos outros eixos

In [ ]:
#from pyspark.sql.functions import avg

In [ ]:
#display(dataframe.select("radius_mean","diagnosis").groupBy("diagnosis").agg(avg("radius_mean")))

In [ ]:
spark.stop()